# Test the Aggregated  Index

## Check if index is up!

In [1]:
import requests
res = requests.get('http://localhost:8055')
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost'], port=8055)
print es

<Elasticsearch([{u'host': 'localhost', u'scheme': 'http'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [389]:
# es.indices.delete(index='agg', ignore=[400, 404])

{u'acknowledged': True}

In [3]:
# es.delete_by_query(index='agg', doc_type='count', body={
#     'query': {
#         "match_all" : {}
#     }
# })

{u'batches': 105,
 u'deleted': 104913,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 1827,
 u'total': 104913,
 u'version_conflicts': 0}

#### Count elements in index

In [3]:
print es.count(index='agg', doc_type='count')
print es.count(index='trend', doc_type='tweet')

{u'count': 158986, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}}
{u'count': 1357550, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}}


In [285]:
result['hits']['total']

194978

#### GET all docs from index

In [398]:
# res = es.search(index="trend", doc_type='tweet', size=10, body={"query": {"match_all": {}}})
res = es.search(index="agg", doc_type='count', size=100000, body={"query": {"match_all": {}}})

# total count 153686
# print "total docs", len(res['hits']['hits'])
# print "total count", sum([x['_source']['count'] for x in res['hits']['hits']])    
lst = []

for r in res['hits']['hits']:
    lst.append(r['_source']['category'])
print list(set(lst))
    
    # break

[u'hamburger', u'guacamole', u'dumplings', u'sushi', u'churros', u'steak', u'macarons', u'falafel', u'gnocchi', u'takoyaki', u'donuts', u'escargots', u'sashimi', u'tiramisu', u'gyoza', u'cheesecake', u'scallops', u'baklava', u'ravioli', u'edamame', u'pho', u'omelette', u'pancakes', u'cannoli', u'tacos', u'ceviche', u'poutine', u'mussels', u'ramen', u'bibimbap', u'bruschetta', u'beignets', u'risotto', u'samosa', u'hummus', u'lasagna', u'paella', u'nachos', u'pizza', u'oysters']


### Max Size Result Windows 

In [1]:
# script for setup
# http://test.tripbuilder.isti.cnr.it:9200/trend/_settings
# { "max_result_window" : 5000000 }
max_result_window = 10000000


### Read file with all the categories

In [9]:
import json
import os
import sys
import time
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
from elasticsearch import Elasticsearch, RequestError
# from processing.preprocess_tweet import process_tweet
# from processing.twitter.Tweet import Tweet
print os.getcwd()


/home/foodmap/food101


In [10]:
dir_file = os.path.join(os.getcwd(), "resources/categories.txt")
categories = []

with open(dir_file, 'r') as f:
    content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    categories = [str.lower(x.strip()) for x in content] 
print "total categories", len(categories)    

total categories 101


### Index agg from index trend

In [306]:
es.delete_by_query(index='agg', doc_type='count', body={
    'query': {
        "match_all" : {}
    }
})

{u'batches': 29,
 u'deleted': 28136,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 334,
 u'total': 28136,
 u'version_conflicts': 0}

In [373]:
# categories
es.delete_by_query(index='agg_day', doc_type='count', body={
    'query': {
        "match_all" : {}
    }
})

{u'batches': 1,
 u'deleted': 284,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 9,
 u'total': 284,
 u'version_conflicts': 0}

### Auxiliar functions

In [7]:
import datetime

# auxiliar functions 
def get_datetime_date(date_int):
    str_date = str(date_int) 
    return datetime.date(int(str_date[0:4]), int(str_date[4:6]), int(str_date[6:8]))    


def get_datetime_month(datet_int):    
    str_date = str(date_int) 
    return datetime.date(int(str_date[0:4]), int(str_date[4:6]), 1)

### Agg - per Day

In [400]:
cat = 'baby back ribs'
cat = str.replace(cat, ' ', '_')
cat

'baby_back_ribs'

In [11]:
numIndex = 1
for cat in categories: 
    # replacing with underscoring
    cat = str.replace(cat, ' ', '_')
    
    # using the index trend
    results_agg = es.search(index='trend', doc_type='tweet', size=max_result_window, body={        
        "size" : max_result_window,
        "query": {
            "bool" : {                
                "should": [
                    {"term" : { "text_categories.keyword" : str(cat) }},
                    {"term" : { "img_categories.keyword" : str(cat) }}
                ]}
        },             
        "aggs": {
            "agg_country": {
              "terms": {
                "field": "country.keyword",
                "size": max_result_window
              },         
              "aggs": {
                "agg_date": {
                  "terms": {
#                    "field": "created_at_datetime",
                    "field": "created_at_day",
                    "size": max_result_window
                  }
                }
              }
            }
        }
    }) 
    
    # create the agg index by day
    for agg_country in results_agg['aggregations']['agg_country']['buckets']:
        country = agg_country['key']
        print cat, country, "count", len(agg_country['agg_date']['buckets']), "doc_error:", results_agg['aggregations']['agg_country']['doc_count_error_upper_bound'] 
        for agg_date in agg_country['agg_date']['buckets']:
            date = agg_date['key']
            count = agg_date['doc_count']            
            
            dic_cout = {
                "category": cat
                , "date": date
                , "country": country
                , "count": count
                , "datetime_date": get_datetime_date(date)
#                 , "datetime_month": get_datetime_month(date)
            }        
            
            es.index(index='agg', doc_type='count', id=numIndex, body=dic_cout)
            numIndex += 1
#    break 
print 'finished!'

apple_pie united states count 140 doc_error: 0
apple_pie united kingdom count 128 doc_error: 0
apple_pie japan count 112 doc_error: 0
apple_pie canada count 91 doc_error: 0
apple_pie france count 47 doc_error: 0
apple_pie italy count 48 doc_error: 0
apple_pie spain count 46 doc_error: 0
apple_pie india count 42 doc_error: 0
apple_pie brazil count 41 doc_error: 0
apple_pie indonesia count 40 doc_error: 0
apple_pie australia count 41 doc_error: 0
apple_pie netherlands count 42 doc_error: 0
apple_pie philippines count 34 doc_error: 0
apple_pie germany count 33 doc_error: 0
apple_pie mexico count 23 doc_error: 0
apple_pie south africa count 21 doc_error: 0
apple_pie argentina count 19 doc_error: 0
apple_pie ireland count 13 doc_error: 0
apple_pie poland count 14 doc_error: 0
apple_pie chile count 12 doc_error: 0
apple_pie colombia count 13 doc_error: 0
apple_pie malaysia count 14 doc_error: 0
apple_pie venezuela count 12 doc_error: 0
apple_pie puerto rico count 13 doc_error: 0
apple_pie un

In [175]:
## debug results 


results = es.search(index='trend', doc_type='tweet', size=max_result_window, body={    
    "query": {
        "bool" : {
            "must": [
                {"term" : { "text_categories" : "pizza" }},
                {"term" : { "country" : "brazil" }}
            ]}
    }
})
print results['hits']['total']
print len(results['hits']['hits'])

idx_result = 90
print results['hits']['hits'][idx_result]['_source']['country'] 
print results['hits']['hits'][idx_result]['_source']['img_categories']
print results['hits']['hits'][idx_result]['_source']['text_categories']
lst = [x['_source']['text_categories'] for x in results['hits']['hits']]

# set(lst)
print len(lst), type(lst) 
list(set([x[0] for x in lst]))


582
582
brazil
None
[u'pizza']
582 <type 'list'>


[u'hot dog', u'apple pie', u'sushi', u'pizza']